In [2]:
mm = require("util.MultiMethod")
return mm, DataType

{new = <function 1>,<metatable> = {__call = <function 1>}}	nil

In [20]:
function dumpfile(fname)
    local f = io.open(fname)
    repeat
        local line = f:read()
        print(line)
    until not line
end

In [141]:
do 
    local proxy_meta = {
        __index = function(self, k)
            local reader = rawget(self, "__reader")
            if reader then
                value = reader(k)
            end
            return rawget(self, "__table")[k]
        end,
        __newindex = function(self, k, v)
            local writer = rawget(self, "__writer")
            if writer then writer(k, v) end
            rawget(self, "__table")[k] = v
            return v
        end,
    }
    TableProxy = function(tbl, reader, writer)
        return setmetatable({
                __table = tbl,
                __reader = reader,
                __writer = writer,
            }, proxy_meta)
    end
end
return TableProxy

<function 1>

In [234]:
do
  local ptype = type
  type = function(x)
    local pt = ptype(x)
    if pt == "table" and x.datatype then
      return x.datatype
    else
      return pt
    end
  end
end

In [240]:
DataType = {byname = {}, datatype = "Module"}

local types = {Type = {datatype = "Type"}}

DataType.typemeta = function(name, abstract)
    local mt = {
        __index = function(self, k)
            return rawget(self, "__table")[k]
        end,
        __newindex = function(...) error("Attempt to modify immutable datatype") end,
        __name = name
    }
    if abstract then
        mt.__call = function(...) error("Can't instantiate abstract data type") end
    end
    return mt
end

DataType.abstract_type = function(self, name)
     if self.types[name] then
        error("Redefine data type")
    end
    local T = {name = name, datatype = "DataType", abstract = true, subtypes = {}}
    types[name] = T
    return T
end

DataType.type = function(self, arg)
    assert(arg and arg.name)
    local name = arg.name
    if self.types[name] then
        print(s)
        error("Redefine data type")
    end
    local constructor = arg.constructror
    local super = nil
    local thetype = {datatype = name, supertype = arg.super}
    if arg.super then
        table.insert(self[arg.super].subtypes, name)
    end
    return thetype
end

DataType.subtypes = function(self, name)
    local supers = {}
    for _, v in pairs(self.types[name].subtypes) do
        table.insert(supers, self[v])
    end
    return supers
end


DataType.supertype = function(self, name)
    return self[self[name].supertype]
end

return DataType

{abstract_type = <function 1>,byname = {},datatype = "Module",subtypes = <function 2>,supertype = <function 3>,type = <function 4>,typemeta = <function 5>}

In [231]:
Shape = DataType:AbstractType("Shape")
print(DataType["Shape"])
return DataType:subtypes("Shape")

nil


{}

In [239]:
Type

nil

In [232]:
Circle = DataType:Type{ name = "Circle", super = "Shape" }
return Shape.subtypes

n/a: (ilua):103: attempt to index a nil value (field '?')

In [233]:
DataType:subtypes("Shape")

{}

In [98]:
local proxy_MT = {
__index = function(self,k)
if self.reading then
self.reading(k)
end
return self.obj[k]
end;
__newindex = function(self,k,v)
if self.writing then
self.writing(k,v)
end
self.obj[k] = v
end
}
 
function table_proxy(o,wfun,rfun)
return setmetatable({
obj = o,
writing = wfun,
reading = rfun
},
proxy_MT)
end

In [100]:
t = table_proxy({x = 1}, print, print)

In [1]:
function dump_raw(data, shift, tag, depth)
    depth = depth == nil and 10 or depth or 0
    local result = ""

    if tag then
        result = result .. tostring(tag) .. " : "
    end

    if type(data) == "table" and depth > 0 then
        shift = (shift or "") .. "  "
        result = result .. tostring(data)
        for k, v in pairs(data) do
            result = result .. "\n" .. shift .. dump_raw(v, shift, k, depth - 1)
        end
    else
        result = result .. tostring(data) .. " (" .. type(data) .. ")"
        if depth == 0 and type(data) == "table" then
            result = result .. " […]"
        end
    end

    return result
end


In [4]:
Square = DataType("Square", function(x) return {size = x} end)
Circle = DataType("Circle", function(x) return {dia = x} end)
s1 = Square(4)
return  s1

<Square 1>

In [6]:
area = mm()
area:addmethod("Square", function(s) return s.size ^ 2 end)
area:addmethod("Circle", function(s) return s.dia ^2 * math.pi end)
return area

{Circle = { <function 1> },Square = { <function 2> },addmethod = <function 3>,<metatable> = {__call = <function 4>}}